In [5]:
import numpy as np
import torch
import h5py

# Using spectral methods to compute the ground truth solution of the PDE
Poisson function is given by:
$-div\ K \nabla u = f$

Helmholtz function is given by:
$- \nabla^2 U + \omega u = f$


Advection-Diffusion function is given by:
$- div\ K\nabla u + v.\nabla u = f$

## Spectral Method with Radial Gaussian Basis Function
Resolution of the grid is $128 \times 128$.\
The given expansion used:\
$$\begin{equation}
f(x) ≈ f^{(N)}(x) = \sum^{N}_{m=1} \phi_i(x)p_i;\ \ \phi_i(x) = \phi(x-x_i)\\
\end{equation}
$$
Here $\phi(x)$ is gaussian with standard diviation $\sigma = \frac{1}{32}$ i.e $4$ times the spatial resolution. Spacing between gaussians is $2\sigma$. $s$ fraction of the $p_i$ value are set to zero and are randomly generated.

We calculate $u$ from $f$ by using by using Fourier Transform of $f$ and then applying the inverse of the required operator and then applying Inverse Fourier Transform to get $u$. To test if our verify the solution, the operator of the equation is applied to $u$ and the norm of it should be very close to $f$

In [6]:
ntrain = 100 # number of training examples
nval = 25 # number of validation examples
ntest = 25 # number of testing examples

nx = ny = 128 # grid size nxn
lx = ly = 1
dx = lx/nx
dy = ly/ny

std = 1/32
space = 2*std
vfs = [0.2, 0.4, 0.6, 0.8]
ng = 144 # number of gaussians in grid
o1 = 1 # sample wavenumbers starting from o1
o2 = 10 # sample wavenumbers starting from o2
diff_coef_scale = 0.01


x = np.arange(0, lx, dx)
y = np.arange(0, ly, dy)

In [7]:
class Grid:
    def __init__(self, nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, isNormalize = True, isRemoval_Intregal =True):
        self.nx = nx
        self.ny = ny
        self.Lx = Lx
        self.Ly = Ly
        self.std = std
        self.space = 2 * self.std
        self.dx = self.Lx / self.nx
        self.dy = self.Ly / self.ny
        self.isNormalize = isNormalize
        self.num_gaussians = num_gaussians
        self.isRemoval_Intregal = isRemoval_Intregal

        x = np.arange(0, self.Lx, self.dx)
        y = np.arange(0, self.Ly, self.dy)
        self.x_cord, self.y_cord = np.meshgrid(x, y) 
        
    def rbf(self, vf = 0.5, center=(0.5, 0.5)):
        # Implementation of radial basis functions
        num_centers_sqrt = np.sqrt(self.num_gaussians)  # Number of centers in each direction
        grid_length = (num_centers_sqrt - 1) * self.space  # Length of the grid in each direction

        # Generate grid of centers
        centers_x = np.arange(center[0] - grid_length / 2, center[0] + grid_length / 2, self.space)
        centers_y = np.arange(center[1] - grid_length / 2, center[1] + grid_length / 2, self.space)

        # Create meshgrid from centers_x and centers_y
        centers_x_grid, centers_y_grid = np.meshgrid(centers_x, centers_y)

        # Calculate distances using vectorized operations
        grid_distance_squared = ((x_cord[..., np.newaxis, np.newaxis] - centers_x_grid)**2 + (y_cord[..., np.newaxis, np.newaxis] - centers_y_grid)**2)

        # Generating a random array and zeroing out x percent of its elements
        p = np.where(np.random.rand(*centers_x_grid.shape) < vf, 0, 
                     0.999 * np.random.rand(*centers_x_grid.shape) + 0.001)

        # Reshape p to have a shape compatible with broadcasting with grid
        p_reshaped = p.reshape(1, 1, *p.shape)

        # Calulate the source functions
        R = 2 * self.std**2
        f_basis_form = np.exp(-grid_distance_squared/R)
        f_grid = np.sum(f_basis_form * p_reshaped, axis=(2, 3))

        # Normalize the basis functions
        if self.isNormalize:
            f_grid /= np.max(f_grid)
            
        # Remove integral of the basis function
        if self.isRemoval_Intregal:
            integral_mean = np.mean(f_grid.flatten())
            f_grid -= integral_mean

        return f_grid

    def fft_coef(self, u, is_torch=False):
        # Implementation of FFT coefficients
        nx, ny = u.shape
        
        if not is_torch:
            ikx = 1j * np.concatenate((np.arange(0, nx//2+1, 1), 
                                       np.arange(-nx//2+1, 0, 1)))
            iky = 1j * np.concatenate((np.arange(0, ny//2+1, 1), 
                                       np.arange(-ny//2+1, 0, 1)))
            
            ikx = ikx.reshape(nx, 1).repeat(ny, axis=1)
            iky = iky.reshape(1, ny).repeat(nx, axis=0)
        else:
            ikx = 1j * torch.cat((torch.arange(0, nx/2+1, 1), 
                                  torch.arange(-nx/2+1, 0, 1)))
            iky = 1j * torch.cat((torch.arange(0, ny/2+1, 1), 
                                  torch.arange(-ny/2+1, 0, 1)))
            
            ikx = np.repeat(ikx.reshape(1, -1), ny, axis=1)
            iky = np.repeat(iky.reshape(-1, 1), nx, axis=0)

        return ikx, iky

    def laplacian(self, u):
        # Implementation of laplacian operator
        ikx, iky = self.fft_coef(u)
        ikx2 = ikx**2
        iky2 = iky**2
        
        u_hat = np.fft.fft2(u)
        u_hat *= (ikx2+iky2) * (4.0 * np.pi**2)/(self.Lx*self.Ly)
        
        return np.real(np.fft.ifft2(u_hat))

    def grad(self, u):
        # Implementation of divergence operator
        ikx, iky = self.fft_coef(u)
        u_hat = np.fft.fft2(u)
        
        ux = np.real(np.fft.ifft2(u_hat * ikx)) * (2.0 * np.pi)/self.Lx
        uy = np.real(np.fft.ifft2(u_hat * iky)) * (2.0 * np.pi)/self.Ly
        
        return ux, uy

    def invLaplacian(self, f, offset=0):
        # Implementation of Inverse laplacian Operator with support for an offset 
        ikx, iky = self.fft_coef(f)
        ikx2 = ikx**2
        iky2 = iky**2
        
        f_hat = np.fft.fft2(f)
        factor = ((ikx2+iky2) * (4.0 * np.pi**2)/(self.Lx*self.Ly)) + offset
        
        factor_zeros = (factor==0)

        u_hat = np.where(factor_zeros, 0, -1 / np.where(factor_zeros, 1, factor)) * np.where(factor_zeros, 0, f_hat)
        
        return np.real(np.fft.ifft2(u_hat))
        
    def invPoission(self, u):
        ikx, iky = self.fft_coef(f)
        ikx2 = ikx**2
        iky2 = iky**2

        f_hat = np.fft.fft2(f)
        factor = ((ikx2+iky2) * (4.0 * np.pi**2)/(self.Lx*self.Ly))

        
        return

In [28]:
a1 = 1
a4 = 5 + np.random.rand() * (5 - 2)

theta = np.random.rand() * 2 * np.pi
rot = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]]) 
theta_neg = -1 * theta
rot_neg = np.array([[np.cos(theta_neg),-np.sin(theta_neg)],[np.sin(theta_neg),np.cos(theta_neg)]])
A = np.array([[a1, 0], [0, a4]])


In [34]:
rot

array([[ 0.96135431,  0.27531417],
       [-0.27531417,  0.96135431]])

In [35]:
rot_neg

array([[ 0.96135431, -0.27531417],
       [ 0.27531417,  0.96135431]])

In [36]:
A 

array([[1.        , 0.        ],
       [0.        , 5.48707492]])

In [33]:
rot_neg @ (A @ rot)

array([[ 1.34011083, -1.18761416],
       [-1.18761416,  5.14696409]])

In [38]:
np.matmul(rot_neg, np.matmul(A, rot))

array([[ 1.34011083, -1.18761416],
       [-1.18761416,  5.14696409]])

In [8]:
mygrid = Grid()
a = mygrid.rbf()
a

array([[ 0.66038786,  0.94274506, -0.05725494, -0.05725494, -0.05725494],
       [-0.05724942, -0.05725494, -0.05725494, -0.05725494, -0.05725494],
       [-0.05725494, -0.05725494, -0.05725494, -0.05725494, -0.05725494],
       [-0.05725494, -0.05725494, -0.05725494, -0.05725494, -0.05725494],
       [-0.05725494, -0.05725494, -0.05725494, -0.05725494, -0.05725494],
       [-0.05725494, -0.05725494, -0.05725494, -0.05725494, -0.05725494]])

In [9]:
alpha = mygrid.laplacian(a)
alpha

array([[-1.00156338e+02, -1.70800841e+02,  4.13825552e+01,
        -1.15833001e+01,  2.64272647e+01],
       [ 5.66616768e+01,  7.89570907e+01, -3.72817511e-05,
        -3.72818067e-05,  2.55533249e-04],
       [-1.88871650e+01, -2.63189451e+01, -4.77518333e-15,
         6.65703482e-15, -6.13734991e-15],
       [ 1.41655556e+01,  1.97392088e+01,  4.73695313e-15,
         2.07241438e-15,  5.32908339e-15],
       [-1.88874923e+01, -2.63189451e+01, -4.69871982e-15,
         2.81686832e-15, -8.07350480e-15],
       [ 5.66626589e+01,  7.89568352e+01, -8.89905670e-16,
        -9.94299031e-15,  1.45079419e-15]])

In [10]:
beta = mygrid.invLaplacian(a)
beta

array([[ 0.00797038,  0.0096367 ,  0.00167155, -0.00120403,  0.00085962],
       [ 0.00296878,  0.00357809,  0.00013024, -0.001667  , -0.00037721],
       [-0.00095851, -0.00082507, -0.00183706, -0.00258863, -0.00204926],
       [-0.00182001, -0.00170127, -0.00249703, -0.00301987, -0.00264466],
       [-0.00095853, -0.00082508, -0.00183706, -0.00258863, -0.00204927],
       [ 0.00296874,  0.00357808,  0.00013023, -0.00166701, -0.00037723]])

In [22]:
class EquationSolver(Grid):
    def __init__(self, equation_type='helmholtz', nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, omega_lower_limt = 0, omega_upper_limit = 1, isNormalize = True, isRemoval_Intregal =True, sparse=False, data_path="./"):
        super().__init__(nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, isNormalize = True, isRemoval_Intregal =True)
        self.equation_type = equation_type
        self.num_gaussians = num_gaussians
        self.data_path = data_path
        self.sparse = sparse
        self.omega_lower_limt = omega_lower_limt
        self.omega_upper_limit = omega_upper_limit

    def helmholtz(self):
        # Implementation for solving Helmholtz equation using Grid methods

        error = 100

        while error > 1E-5:
            source = self.rbf()
            omega = np.random.randint(self.omega_lower_limt, self.omega_upper_limit+1)
            
            u_self = self.invLaplacian(source, omega)
            lhs = (self.laplacian(u_self) + omega*u_self)
            
            error = np.abs(np.linalg.norm(lhs)-np.linalg.norm(source))
            if (error > 1E-5):
                print("Didn't converge error at ", error)
        
        return u_self, source, omega

    def poisson(self):
        # Implementation for solving Poisson equation using Grid methods
        
        return u, source, k

    def advection_diffusion(self):
        # Implementation for solving Advection-Diffusion equation using Grid methods
        return u, source, diffusion_tensor

    def create_hdf5(self, path, dat, ten):
        # Implementation to create HDF5 files
        pass

    def generate_data(self, ntrain=100, nval=25, ntest=25, e1=1, e2=5, o1=1, o2=10):
        train = np.zeros((ntrain, 2, self.grid_size, self.grid_size))
        train_info = np.zeros((ntrain, 2))
        val = np.zeros((nval, 2, self.grid_size, self.grid_size))
        val_info = np.zeros((nval, 2))
        test = np.zeros((ntest, 2, self.grid_size, self.grid_size))
        test_info = np.zeros((ntest, 2))

        for idx, vf in enumerate([0.2, 0.4, 0.6, 0.8]):
            if self.equation_type == 'helmholtz':
                u, source, info = self.helmholtz()
            elif self.equation_type == 'poisson':
                u, source, info = self.poisson()
            elif self.equation_type == 'advdiff':
                u, source, info = self.advection_diffusion()

            # Assign data to appropriate datasets (train, validation, test)

        # Save data into HDF5 files


In [26]:
helmholtz_eq = EquationSolver(equation_type='helmholtz', isRemoval_Intregal = False)
helmholtz_sol, helmholtz_f, helmholtz_omega = helmholtz_eq.helmholtz()
print("Helmholtz, u:", helmholtz_sol)
print("f:", helmholtz_sol)
print("omega:", helmholtz_omega)

Helmholtz, u: [[ 0.00908527  0.0059224   0.00030816 -0.00102628  0.00184932]
 [ 0.00336849  0.00221186 -0.00058687 -0.0014209   0.00037636]
 [-0.00063347 -0.00088678 -0.00185769 -0.00220646 -0.00145488]
 [-0.00138801 -0.00161341 -0.00233141 -0.00257403 -0.00205119]
 [-0.0006335  -0.00088679 -0.00185769 -0.00220647 -0.00145489]
 [ 0.00336844  0.00221184 -0.00058688 -0.0014209   0.00037634]]
f: [[ 0.00908527  0.0059224   0.00030816 -0.00102628  0.00184932]
 [ 0.00336849  0.00221186 -0.00058687 -0.0014209   0.00037636]
 [-0.00063347 -0.00088678 -0.00185769 -0.00220646 -0.00145488]
 [-0.00138801 -0.00161341 -0.00233141 -0.00257403 -0.00205119]
 [-0.0006335  -0.00088679 -0.00185769 -0.00220647 -0.00145489]
 [ 0.00336844  0.00221184 -0.00058688 -0.0014209   0.00037634]]
omega: 0
